<a href="https://colab.research.google.com/github/SHASHANKV9611/NLP_sentiment_analysis/blob/main/NLP_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
data= pd.read_csv('/content/drive/MyDrive/Internship/NLP/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [ ]:
import re
from bs4 import BeautifulSoup
def remove_noise(text):
  soup=BeautifulSoup(text,"html.parser")
  text=soup.get_text()
  text=re.sub('\[[^]]*\]','',text)
  return text

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data['review']=data['review'].apply(remove_noise)

<ipython-input-6-595fe67b42db>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup=BeautifulSoup(text,"html.parser")


In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


#Stemming

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
def stemmer(text):
  ps=nltk.porter.PorterStemmer()
  text=' '.join([ps.stem(word) for word in text.split()])
  return text

In [ ]:
#Applying the stemmer for the review column

data['review']=data['review'].apply(stemmer)

In [ ]:
data.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,a wonder littl production. the film techniqu i...,positive
2,i thought thi wa a wonder way to spend time on...,positive
3,basic there' a famili where a littl boy (jake)...,negative
4,"petter mattei' ""love in the time of money"" is ...",positive


#Removing Stop words

In [ ]:
#importing the stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_wr=set(nltk.corpus.stopwords.words('english'))
len(stop_wr)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
# creating an function to remove stop words

def remove_stopwords(text,is_lower_case=False):
  tokenizers = ToktokTokenizer()
  tokens=tokenizers.tokenize(text)
  tokens=[i.strip() for i in tokens]
  if is_lower_case:
    filtokens=[i for i in tokens if token not in stop_wr]
  else:
    filtokens=[i for i in tokens if i.lower() not in stop_wr]
  filtered_text=' '.join(filtokens)
  return filtered_text

In [ ]:
data['review']=data['review'].apply(remove_stopwords)

In [ ]:
data.head()

,review,sentiment
0,one review ha mention watch 1 oz episod ' hook...,positive
1,wonder littl production. film techniqu veri un...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic ' famili littl boy ( jake ) think ' zomb...,negative
4,"petter mattei ' "" love time money "" visual stu...",positive


#Train Test Split

In [ ]:
train_data=data.review[:30000]
test_data=data.review[30000:]

#Bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
cv_train=cv.fit_transform(train_data)
cv_test=cv.transform(test_data)

print("BOW train shape: ",cv_train.shape)
print("BOW test shape: ",cv_test.shape)

BOW train shape:  (30000, 4954557)
BOW test shape:  (20000, 4954557)


#TFIDF

In [ ]:
tf=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
tf_train=tf.fit_transform(train_data)
tf_test=tf.transform(test_data)

print("TFIDF train shape: ",tf_train.shape)
print("TFIDF test shape: ",tf_test.shape)

TFIDF train shape:  (30000, 4954557)
TFIDF test shape:  (20000, 4954557)


#Label Encoding

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label=LabelBinarizer()
sentiment_data=label.fit_transform(data['sentiment'])
print(sentiment_data)

[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


In [ ]:
#splitting

train_sdata=data.sentiment[:30000]
test_sdata=data.sentiment[30000:]

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
lr_bow=lr.fit(cv_train,train_sdata)
print(lr_bow)

LogisticRegression(C=1, max_iter=500, random_state=42)


In [ ]:
bow_pred=lr.predict(cv_test)
print(bow_pred)

['negative' 'negative' 'negative' ... 'negative' 'negative' 'negative']


In [ ]:
from nltk import accuracy
from sklearn.metrics import accuracy_score

bow_score=accuracy_score(bow_pred,test_sdata)
print(bow_score)

0.59215


In [ ]:
lrtf=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
lr_tf=lrtf.fit(tf_train,train_sdata)

In [ ]:
tf_pred=lrtf.predict(tf_test)
print(tf_pred)

['negative' 'negative' 'negative' ... 'negative' 'positive' 'positive']


In [ ]:
tf_score=accuracy_score(test_sdata,tf_pred)
print(tf_score)

0.7426
